In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# open Posts.csv file
df = pd.read_csv('./data/Posts.csv')

# workaround for que 2
# df.loc[:, 'Tags'] = df['Tags'].fillna('|darpan|')

In [3]:
df.head()

,Id,OwnerUserId,LastEditorUserId,PostTypeId,AcceptedAnswerId,Score,ParentId,ViewCount,AnswerCount,CommentCount,...,Title,Tags,ContentLicense,Body,FavoriteCount,CreationDate,CommunityOwnedDate,ClosedDate,LastEditDate,LastActivityDate
0,1,6.0,226.0,1,13.0,99,NaN,65597.0,34.0,10,...,"""Comments are a code smell""",|comments|anti-patterns|,CC BY-SA 2.5,<p>A coworker of mine believes that <em>any</e...,NaN,2010-09-01T19:34:48.000,2011-01-31T09:04:54.130,2012-11-27T20:11:51.580,2011-11-25T22:32:41.300,2012-11-27T19:29:27.740
1,3,11.0,11.0,2,NaN,29,1.0,NaN,NaN,17,...,NaN,NaN,CC BY-SA 2.5,"<p>Ideally, code should be so well coded that ...",NaN,2010-09-01T19:36:50.053,2011-01-31T09:04:54.130,NaN,2010-09-01T20:41:14.273,2010-09-01T20:41:14.273
2,4,NaN,NaN,1,26.0,66,NaN,10023.0,12.0,2,...,Getting non-programmers to understand the deve...,|business|project-management|development-process|,CC BY-SA 2.5,<p>When starting a project for a company that'...,NaN,2010-09-01T19:37:39.957,2022-11-03T13:52:50.803,NaN,2010-09-01T19:45:26.117,2013-03-20T19:59:57.770
3,7,21.0,NaN,2,NaN,10,1.0,NaN,NaN,0,...,NaN,NaN,CC BY-SA 2.5,"<p>I think the answer is the usual ""It depends...",NaN,2010-09-01T19:42:16.797,2011-01-31T09:04:54.130,NaN,NaN,2010-09-01T19:42:16.797
4,9,17.0,666.0,1,NaN,39,NaN,13944.0,24.0,1,...,How do you stay focused at work?,|productivity|time-management|,CC BY-SA 2.5,"<p>Sometimes, the things I have to do for my j...",NaN,2010-09-01T19:43:04.957,2011-01-04T21:22:43.957,2012-11-13T19:09:35.853,2010-09-11T09:35:35.957,2011-01-04T21:22:43.957


## Exercise 1

In [4]:
# make table containing unique answerers and number of questions answered by them
answers = df[df['PostTypeId'] == 2]

answerers = answers.groupby('OwnerUserId').size().reset_index(name='counts')

answerers = answerers.sort_values(by='counts', ascending=False)

# print top 3 answerers
print("Top 3 answerers:")
print(answerers.head(3))

Top 3 answerers:
       OwnerUserId  counts
3189        9113.0    2839
19912     177980.0    2326
557         1204.0    2043


In [5]:
# make table containing tag and number of questions with that tag
questions = df[df['PostTypeId'] == 1]

# tags are of form |tag1|tag2|...|tagN|
# split tags into list
# questions['Tags'] = questions['Tags'].str.split('|')
# use .loc to avoid SettingWithCopyWarning
questions.loc[:, 'Tags'] = questions['Tags'].str.split('|')

# explode tags into separate rows
tags = questions.explode('Tags')

# remove empty tags
# tags = tags[tags['Tags'] != '']

tags = tags.groupby('Tags').size().reset_index(name='counts')

tags = tags.sort_values(by='counts', ascending=False)

# print top 3 tags
print("Top 3 tags:")
print(tags.head())

Top 3 tags:
                Tags  counts
0                     126846
416           design    5162
188               c#    4931
774             java    4928
418  design-patterns    4449


## Exercise 2

In [6]:
'''
In this exercise, you will create a utility matrix for experts, who are answerers
with some threshold answer count. Create a utility matrix Experts, where the rows are the answerers who
have answered at least 20 questions in the Answerer table and and the columns are the tags in the Tags
table that were annotated at least 20 times. In this step, the goal is to remove answerers who have
answered very few questions or the tags that have been used rarely. Each entry in the Expert matrix
should contain the count of the number of posts an expert has answered on the particular tag. Sort the
rows and columns of the Expert matrix in the ascending order of the ids.
'''

# filter answerers with at least 20 answers
answerers = answerers[answerers['counts'] >= 20]
print(answerers.shape)

# filter tags with at least 20 questions
tags = tags[tags['counts'] >= 20]
print(tags.shape)

# # get posts of answerers with at least 20 answers
# ans2 = answers[answers['OwnerUserId'].isin(answerers['OwnerUserId'])]

# get posts of questions with tags that have at least 20 questions
# here tag in tags is sustring of tag in questions
# so we need to check if tag in tags is substring of tag in questions
# we can use apply and lambda function to do this
# posts -> where tags is not null and tag in tags is substring of tag in questions
# change null tag to |darpan| in df
posts = df.copy()


# get posts of answerers with at least 20 answers


# print uniue owner user ids
# print(posts['OwnerUserId'].nunique())

# explode tags into separate rows
posts.loc[:, 'Tags'] = posts['Tags'].str.split('|')
posts = posts.explode('Tags')

# get posts of questions with tags that have at least 20 questions
# posts = posts[posts['Tags'].isin(tags['Tags'])]
# posts = posts[posts['OwnerUserId'].isin(answerers['OwnerUserId'])]

posts = df[df['OwnerUserId'].isin(answerers['OwnerUserId'])]
print(posts['OwnerUserId'].nunique())
print(posts['Tags'].nunique())

(1163, 2)
(975, 2)
1163
3961


In [7]:
# group by answerer and tag
utility = posts.groupby(['OwnerUserId', 'Tags']).size().reset_index(name='counts')

# pivot table
utility = utility.pivot(index='OwnerUserId', columns='Tags', values='counts')

# fill NaN with 0
utility = utility.fillna(0)

# print dimensions of utility matrix
print("Dimensions of utility matrix:")
print(utility.shape)

utility.head()

Dimensions of utility matrix:
(706, 3961)


Tags,|.net|,|.net|.net-core|,|.net|android|app|windows-8|tablets|,|.net|api|mobile|http-response|,|.net|applications|access|,|.net|architecture|soa|services|coupling|,|.net|asp.net|performance|iis|jit|,|.net|bcl|extension-method|,|.net|c#|object-oriented|,|.net|code-generation|,...,|writing|technical-writing|,|xml|,|xml|big-data|document-databases|,|xml|continuous-integration|configuration|continuous-delivery|settings|,|xml|http|namespace|uri|,|xml|json|,|xml|lisp|linq|xslt|,|xml|naming|,|xml|windows-registry|,|xml|xsd|
OwnerUserId,,,,,,,,,,,,,,,,,,,,,
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Exercise 2

In [8]:
posts = df.copy()

In [9]:
questions = posts[posts['PostTypeId'] == 1][['Id', 'Tags']]

answers = posts[posts['PostTypeId'] == 2][['OwnerUserId', 'ParentId']].dropna(subset=['OwnerUserId'])
answers['OwnerUserId'] = answers['OwnerUserId'].astype(int)

answerer_counts = answers.groupby('OwnerUserId').size().reset_index(name='AnswerCount')
expert_answerers = answerer_counts[answerer_counts['AnswerCount'] >= 20]['OwnerUserId']

questions['Tags'] = questions['Tags'].fillna('').str.strip('|').str.split('|')
exploded_tags = questions.explode('Tags')

tag_counts = exploded_tags.groupby('Tags').size().reset_index(name='TagCount')
popular_tags = tag_counts[tag_counts['TagCount'] >= 20]['Tags']

answerer_tags = pd.merge(answers, exploded_tags, left_on='ParentId', right_on='Id', how='left')

filtered_data = answerer_tags[
    (answerer_tags['OwnerUserId'].isin(expert_answerers)) &
    (answerer_tags['Tags'].isin(popular_tags))
]

utility_matrix = pd.pivot_table(
    filtered_data, index='OwnerUserId', columns='Tags', aggfunc='size', fill_value=0
)

utility_matrix.sort_index(inplace=True)
utility_matrix.sort_index(axis=1, inplace=True)

print("Number of expert answerers (>=20 answers):", len(expert_answerers))
print("Number of popular tags (>=20 annotations):", len(popular_tags))

print("\nDimension of the Expert utility matrix:")
print(utility_matrix.shape)

print("\nUtility Matrix:")
print(utility_matrix)

Number of expert answerers (>=20 answers): 1163
Number of popular tags (>=20 annotations): 974

Dimension of the Expert utility matrix:
(1163, 974)

Utility Matrix:
Tags         .net  .net-core  3d  64-bit  abstract-class  abstract-factory  \
OwnerUserId                                                                  
4               6          0   0       0               0                 1   
6               0          0   0       0               0                 0   
11              1          0   0       0               0                 0   
14              0          0   0       0               0                 0   
15              0          0   0       0               0                 0   
...           ...        ...  ..     ...             ...               ...   
356695          1          0   0       0               0                 0   
366014          0          0   0       0               0                 0   
373864          0          0   0       0               

## Exercise 3

In [10]:
# new utitlity matrix where rij = cij/3 if cij<15 else 5
utility_matrix_new = utility_matrix.copy()
utility_matrix_new = utility_matrix_new.map(lambda x: x//3 if x < 15 else 5)

In [11]:
# Make test data, bottom 15% experts and rightmost 15% tags

test_data = utility_matrix_new.iloc[-int(0.15*utility_matrix_new.shape[0]):, -int(0.15*utility_matrix_new.shape[1]):]
train_data = utility_matrix_new.iloc[:-int(0.15*utility_matrix_new.shape[0]), :-int(0.15*utility_matrix_new.shape[1])]

In [12]:
# get utility matrix sum
utility_matrix_sum = utility_matrix_new.sum().sum()

# highest row sum of utility matrix
highest_row_sum = utility_matrix_new.sum(axis=1).max()

# highest column sum of utility matrix
highest_column_sum = utility_matrix_new.sum(axis=0).max()

# print utility matrix sum, highest row sum and highest column sum
print("Utility matrix sum:", utility_matrix_sum)
print("Highest row sum:", highest_row_sum)
print("Highest column sum:", highest_column_sum)

Utility matrix sum: 41403
Highest row sum: 1164
Highest column sum: 1403


In [13]:
# summation value of the train data
train_data_sum = train_data.sum().sum()

# dimension of the test data
test_data_dimension = test_data.shape

# summation value of the test data
test_data_sum = test_data.sum().sum()

# print summation value of the train data, dimension of the test data and summation value of the test data
print("Summation value of the train data:", train_data_sum)
print("Dimension of the test data:", test_data_dimension)
print("Summation value of the test data:", test_data_sum)

Summation value of the train data: 28713
Dimension of the test data: (174, 146)
Summation value of the test data: 899


In [14]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import numpy as np

In [17]:
def calculate_item_similarity(matrix):
    #  use pearson correlation to calculate similarity between items
    return matrix.T.corr(method='pearson')

item_similarity_matrix = calculate_item_similarity(utility_matrix_new)

user_similarity_matrix = utility_matrix_new.corr(method='pearson')

def predict_rating_simple_average(user_id, tag_id, N):
    if tag_id not in item_similarity_matrix.columns:
        return 0
    similar_tags = item_similarity_matrix[tag_id].sort_values(ascending=False).index[1:N+1]
    ratings = utility_matrix_new.loc[user_id, similar_tags]
    return ratings.mean() if not ratings.empty else 0

def predict_rating_weighted_average(user_id, tag_id, N):
    if tag_id not in item_similarity_matrix.columns:
        return 0
    similar_tags = item_similarity_matrix[tag_id].sort_values(ascending=False).index[1:N+1]
    ratings = utility_matrix_new.loc[user_id, similar_tags]
    weights = item_similarity_matrix[tag_id][similar_tags] 
    valid_ratings = ratings.notnull()
    weighted_sum = (ratings[valid_ratings] * weights[valid_ratings]).sum()
    weight_sum = weights[valid_ratings].sum()
    return weighted_sum / weight_sum if weight_sum != 0 else 0

def predict_rating_user_simple_average(user_id, tag_id, N):
    if user_id not in user_similarity_matrix.columns:
        return 0
    if tag_id not in utility_matrix_new.columns:
        return 0
    similar_users = user_similarity_matrix[user_id].sort_values(ascending=False).index[1:N+1]
    ratings = utility_matrix_new.loc[similar_users, tag_id]
    return ratings.mean() if not ratings.empty else 0

def predict_rating_user_weighted_average(user_id, tag_id, N):
    if user_id not in user_similarity_matrix.columns:
        return 0
    if tag_id not in utility_matrix_new.columns:
        return 0
    similar = user_similarity_matrix[user_id].sort_values(ascending=False).index[1:N+1]
    ratings = utility_matrix_new.loc[similar, tag_id]
    weights = user_similarity_matrix[user_id][similar]
    valid_ratings = ratings.notnull()
    weighted_sum = (ratings[valid_ratings] * weights[valid_ratings]).sum()
    weight_sum = weights[valid_ratings].sum()
    return weighted_sum / weight_sum if weight_sum != 0 else 0

def evaluate_rmse(predict_function, N):
    actual_ratings = []
    predicted_ratings = []

    for user_id in test_data.index:
        for tag_id in test_data.columns:
            if not pd.isnull(test_data.loc[user_id, tag_id]):
                actual_ratings.append(test_data.loc[user_id, tag_id])
                predicted_ratings.append(predict_function(user_id, tag_id, N))

    return np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))

def evaluate_rmse_user(predict_function, N):
    actual_ratings = []
    predicted_ratings = []

    for user_id in test_data.index:
        for tag_id in test_data.columns:
            if not pd.isnull(test_data.loc[user_id, tag_id]):
                actual_ratings.append(test_data.loc[user_id, tag_id])
                predicted_ratings.append(predict_function(user_id, tag_id, N))

    return np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))

# Test Evaluation and Display Results
def display_results():
    N_values = [2, 3, 5]

    print("Method | Rating Prediction Function | Metric | N=2 | N=3 | N=5")
    print("------------------------------------------------------------")

    for method, predict_simple, predict_weighted in [
        ("Item-Item", predict_rating_simple_average, predict_rating_weighted_average),
        ("User-User", predict_rating_user_simple_average, predict_rating_user_weighted_average)
    ]:
        print(f"{method} | Simple Average | RMSE |", end=" ")
        for N in N_values:
            rmse = evaluate_rmse(predict_simple, N) if method == "Item-Item" else evaluate_rmse_user(predict_simple, N)
            print(f"{rmse:.4f} |", end=" ")
        print()

        print(f"{method} | Weighted Average | RMSE |", end=" ")
        for N in N_values:
            rmse = evaluate_rmse(predict_weighted, N) if method == "Item-Item" else evaluate_rmse_user(predict_weighted, N)
            print(f"{rmse:.4f} |", end=" ")
        print()

# Call the function to display results
display_results()


Method | Rating Prediction Function | Metric | N=2 | N=3 | N=5
------------------------------------------------------------
Item-Item | Simple Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 
Item-Item | Weighted Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 
User-User | Simple Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 
User-User | Weighted Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 


In [18]:
from sklearn.metrics import mean_squared_error

# Assuming the utility_matrix is already defined as a NumPy array or pandas DataFrame
regularization_params = [(0.001, 0.003), (0.05, 0.05), (0.50, 0.75)]
K_values = [2, 5, 10]
num_iterations = 1000  # Reduced number of iterations
learning_rate = 0.0005
tolerance = 1e-4  # Early stopping criterion

# Define RMSE calculation function
def calculate_rmse(user_item_matrix, P, Q):
    predictions = np.dot(P, Q.T)
    actual = user_item_matrix[user_item_matrix > 0]  # Only non-zero ratings
    predicted = predictions[user_item_matrix > 0]  # Corresponding predictions
    return np.sqrt(mean_squared_error(actual, predicted))

# Define optimized SGD with and without regularization
def sgd(utility_matrix, P, Q, learning_rate, lambda1=0, lambda2=0):
    num_users, num_items = utility_matrix.shape

    for iteration in range(num_iterations):
        # Vectorized dot product for predictions
        predictions = np.dot(P, Q.T)
        error_matrix = (utility_matrix > 0) * (utility_matrix - predictions)

        # Update P and Q
        P += learning_rate * (np.dot(error_matrix, Q) - lambda1 * P)
        Q += learning_rate * (np.dot(error_matrix.T, P) - lambda2 * Q)

        # Optional: check for convergence (if error is reducing)
        cost = np.linalg.norm((utility_matrix - np.dot(P, Q.T)) * (utility_matrix > 0))
        if cost < tolerance:
            break

    return P, Q

# DataFrame to store results
results_df = pd.DataFrame(columns=["Method", "Metric", "K=2", "K=5", "K=10"])

# Loop through different K values and regularization settings
for regularized, (lambda1, lambda2) in [(False, (0, 0))] + [(True, l1_l2) for l1_l2 in regularization_params]:
    for K in K_values:
        # Initialize P and Q matrices
        num_users, num_items = utility_matrix_new.shape
        P = np.random.rand(num_users, K)
        Q = np.random.rand(num_items, K)

        # Train the model
        P, Q = sgd(utility_matrix_new.values, P, Q, learning_rate, lambda1, lambda2)

        # Calculate RMSE after training
        rmse = calculate_rmse(utility_matrix_new.values, P, Q)

        # Add the results to the DataFrame
        if not regularized:
            results_df.loc[len(results_df)] = ["Without Regularisation", "RMSE", rmse if K == 2 else "", rmse if K == 5 else "", rmse if K == 10 else ""]
        else:
            method = f"With Regularisation λ1={lambda1}, λ2={lambda2}"
            results_df.loc[len(results_df)] = [method, "RMSE", rmse if K == 2 else "", rmse if K == 5 else "", rmse if K == 10 else ""]

# Display the results
print(results_df)


                                    Method Metric       K=2       K=5  \
0                   Without Regularisation   RMSE  0.875118             
1                   Without Regularisation   RMSE            0.748425   
2                   Without Regularisation   RMSE                       
3   With Regularisation λ1=0.001, λ2=0.003   RMSE  0.876595             
4   With Regularisation λ1=0.001, λ2=0.003   RMSE            0.740916   
5   With Regularisation λ1=0.001, λ2=0.003   RMSE                       
6     With Regularisation λ1=0.05, λ2=0.05   RMSE  0.878973             
7     With Regularisation λ1=0.05, λ2=0.05   RMSE            0.736233   
8     With Regularisation λ1=0.05, λ2=0.05   RMSE                       
9      With Regularisation λ1=0.5, λ2=0.75   RMSE  0.881022             
10     With Regularisation λ1=0.5, λ2=0.75   RMSE            0.752315   
11     With Regularisation λ1=0.5, λ2=0.75   RMSE                       

        K=10  
0             
1             
2    

## Exercise 6

In [ ]:
# Surprise Library
